# Data Profile
Copyright (c) Microsoft Corporation. All rights reserved.<br>
Licensed under the MIT License.

A DataProfile collects summary statistics on each column of the data produced by a Dataflow. This can be used to:
- Understand the input data.
- Determine which columns might need further preparation.
- Verify that data preparation operations produced the desired result.

`Dataflow.get_profile()` executes the Dataflow, calculates profile information, and returns a newly constructed DataProfile.

In [ ]:
import azureml.dataprep as dprep

dflow = dprep.auto_read_file('../data/crime-spring.csv')

profile = dflow.get_profile()
profile

A DataProfile contains a collection of ColumnProfiles, indexed by column name. Each ColumnProfile has attributes for the calculated column statistics. For non-numeric columns, profiles include only basic statistics like min, max, and error count. For numeric columns, profiles also include statistical moments and estimated quantiles.

In [ ]:
profile.columns['Beat']

You can also extract and filter data from profiles by using list and dict comprehensions.

In [ ]:
variances = [c.variance for c in profile.columns.values() if c.variance]
variances

In [ ]:
column_types = {c.name: c.type for c in profile.columns.values()}
column_types

If a column has fewer than a thousand unique values, its ColumnProfile contains a summary of values with their respective counts.

In [ ]:
profile.columns['Primary Type'].value_counts

Numeric ColumnProfiles include an estimated histogram of the data.

In [ ]:
profile.columns['District'].histogram

To configure the number of bins in the histogram, you can pass an integer as the `number_of_histogram_bins` parameter.

In [ ]:
profile_more_bins = dflow.get_profile(number_of_histogram_bins=5)
profile_more_bins.columns['District'].histogram

For columns containing data of mixed types, the ColumnProfile also provides counts of each type.

In [ ]:
profile.columns['X Coordinate'].type_counts

In [ ]:
#TEST CELL: Profile.Compare
import azureml.dataprep as dprep
import math

lhs_dflow = dprep.auto_read_file('../data/crime-spring.csv')
lhs_profile = lhs_dflow.get_profile(number_of_histogram_bins=100)
rhs_dflow = dprep.auto_read_file('../data/crime-winter.csv')
rhs_profile = rhs_dflow.get_profile(number_of_histogram_bins=100)

diff = lhs_profile.compare(rhs_profile)

expected_col1 = dprep.ColumnProfileDifference()
expected_col1.difference_in_count_in_percent = 0
expected_col1.difference_in_histograms = 135349.66146244822

for actual, expected in zip(diff.column_profile_difference, [expected_col1]) :
    assert math.isclose(actual.difference_in_count_in_percent, expected.difference_in_count_in_percent)
    assert math.isclose(actual.difference_in_histograms, expected.difference_in_histograms)
    break
